# Summary
1. Here, we create unnatural bareground layers. We will consider annual rainfal and classify pixels that are not green enough as bare ground.
2. We consider annual rainfall data by combining CHIRPS Daily data.
3. We select for each pixel the wettest month for the year
4. Using Sentinel-2 we select the four greenest months of imagery and calculate the NDVI
5. Using a log threshold, we then classify low NDVI values as bareground

### Future
- Exporting to Google Cloud

# Setting up the environment

In [ ]:
# Import and/or install libraries
import subprocess, os, gcsfs, json

try:
    import geemap, ee
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', '-U', 'geemap'])
    import geemap, ee


In [ ]:
# Connect to Google Drive to access files

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Connect to Google Earth Engine if neccessary

service_account = os.environ.get('GOOGLE_SERVICE_ACCOUNT')
credentials = ee.ServiceAccountCredentials(service_account, os.environ.get('GOOGLE_APPLICATION_CREDENTIALS'))
ee.Initialize(credentials)

# Sentinel

In [ ]:
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50


def get_s2_sr_cld_col(datefilter):
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filter(datefilter)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filter(datefilter)
        )

    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))


def add_cloud_bands(img):
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')
    return img.addBands(ee.Image([cld_prb, is_cloud]))


def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)


def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)


date_filter = ee.Filter.date(ee.Date('2022-01-01'), ee.Date('2023-01-01'))

sentinel = get_s2_sr_cld_col(date_filter).map(add_cld_shdw_mask).map(apply_cld_shdw_mask)





# Rain

In [ ]:
start_year = 2022
end_year = start_year + 1

# Define the initial ImageCollection
rain = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filter(ee.Filter.date(f'{start_year}-01-01', f'{end_year}-01-01')).select('precipitation')

# Annual rainfall
annual_rainfall = rain.sum()

# Create an ee.List of months
months = ee.List.sequence(1, 12)

# Function to calculate the sum of precipitation for each month, add the month as a band, and return the resulting image
def calculate_monthly_sum(month):
    month = ee.Number(month)
    start_date = ee.Date.fromYMD(start_year, month, 1)
    end_date = start_date.advance(1, 'month')
    rain_month = rain.filterDate(start_date, end_date)
    sum_month = rain_month.reduce(ee.Reducer.sum()).rename('precipitation')
    sum_month = sum_month.addBands(ee.Image.constant(month).toInt().rename('month'))
    return sum_month.set('year', start_year).set('month', month)

monthly_rain = months.map(calculate_monthly_sum)
monthly_rain = ee.ImageCollection(monthly_rain)

# Create a quality mosaic
wettest_month = monthly_rain.qualityMosaic('precipitation')

# Select the month band
wettest_month = wettest_month.select('month')

# Create a month mask
def mask_wettest_month(month):
    month = ee.Number(month)
    month_mask = wettest_month.eq(month).selfMask().set('month', month)
    return month_mask

month_list = ee.List.sequence(1, 12)
month_images = ee.ImageCollection(month_list.map(mask_wettest_month))


# Sentinel: greenest months


In [ ]:
import math

CLOUD_FILTER = 40
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

start_date = ee.Date.fromYMD(start_year, 1, 1)
end_date = start_date.advance(1, 'month')

def calc_ndvi(image):
  ndvi = image.normalizedDifference(['B8', 'B4'])
  return ndvi


def get_ndvi_for_month(image):
  start_date = ee.Date.fromYMD(start_year, image.get('month'), 1)
  start_date_ndvi = start_date.advance(-15, 'day')
  date_filter = ee.Filter.date(ee.Date(start_date_ndvi), ee.Date(start_date_ndvi).advance(4, 'month'))

  sentinel = get_s2_sr_cld_col(date_filter).map(add_cld_shdw_mask).map(apply_cld_shdw_mask)
  ndvi = sentinel.map(calc_ndvi).max().updateMask(image)

  return ndvi

ndvi_for_wettest_month = month_images.map(get_ndvi_for_month).max()


In [ ]:
print(annual_rainfall.bandNames().getInfo())

# Calculate bareground

In [ ]:
threshold = annual_rainfall.expression(
    'log10(PREC-100)**0.53-1.43', {
      'PREC': annual_rainfall.select('precipitation')
})
bareground = ndvi_for_wettest_month.updateMask(ndvi_for_wettest_month.lt(threshold))


# Map

In [ ]:
ndvi_style = {
  'min': 0.0,
  'max': 1.0,
  'palette': ['8C510A', 'D8B365', 'F6E8C3', 'a7d07b', '5bc02f', '2fbb00']

}

rain_highest_style = {
    'min': 1,
    'max': 12,
    'palette': ['#FF0000', '#FF7F00', '#FFFF00', '#7FFF00', '#00FF00', '#00FF7F', '#00FFFF', '#007FFF', '#0000FF', '#7F00FF', '#FF00FF', '#FF007F']

}

sentinel_vis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2']
}

Map = geemap.Map()
Map.add_basemap('SATELLITE')
# Map.addLayer(wettest_month, rain_highest_style, 'highest_rainfall_month')
# Map.addLayer(ndvi_for_wettest_month, ndvi_style, 'ndvi_for_wettest_month')
Map.addLayer(bareground, {}, 'bareground')


# Map.addLayer(sentinel, sentinel_vis, 'sentinel')
Map.setCenter(31.944, -25.087, 12)
Map